# Setfit few shot learning

In [ ]:
#!pip install setfit
https://www.youtube.com/watch?v=Pg-smN4fUy0

In [75]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer

## Load custom dataset

In [76]:
from sklearn.model_selection import train_test_split
train_dataset = load_dataset('csv', data_files='data/smabuse.csv', split='train')
eval_dataset = train_dataset.train_test_split(test_size=0.1)


Using custom data configuration default-bc51b014c530ee1e
Found cached dataset csv (/home/gary_hutson/.cache/huggingface/datasets/csv/default-bc51b014c530ee1e/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


In [77]:
# Create Dataset dict
from datasets import DatasetDict
dataset = DatasetDict({
    'train': train_dataset, 
    'validation': eval_dataset['test']
})
print(dataset)
print(dataset['validation']['text'])


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 22
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 3
    })
})
['They have to be the ugliest person I have ever seen on screen', 'You look so fat in that picture', "People suffer from body issues and that's okay"]


# Push our custom dataset to the hub

In [78]:
from huggingface_hub import notebook_login
notebook_login()

In [79]:
dataset.push_to_hub('socialmedia-abuse')

Pushing split train to the Hub.
Resuming upload of the dataset shards.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split validation to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

## Pull dataset from HuggingFace hub

In [80]:
dataset = load_dataset('StatsGary/socialmedia-abuse')
print(dataset)

Using custom data configuration StatsGary--socialmedia-abuse-8383fb3795d399f9


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/22 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /home/gary_hutson/.cache/huggingface/datasets/StatsGary___parquet/StatsGary--socialmedia-abuse-8383fb3795d399f9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 22
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 3
    })
})


## Map the encoding to a label

In [102]:
id2label = {0: "Not Abuse", 1: "Abuse"}
label2id = {"Not Abuse": 0, "Abuse": 1}

## Simulate few shot learning by sampling 8 examples per class

In [83]:
num_class = 2
train_ds = dataset['train'].shuffle(seed=123).select(range(8 * num_class))
eval_ds = dataset['validation']

Loading cached shuffled indices for dataset at /home/gary_hutson/.cache/huggingface/datasets/StatsGary___parquet/StatsGary--socialmedia-abuse-8383fb3795d399f9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-53b8aaa66e353a4d.arrow


In [84]:
train_ds['text']

['Your appearance makes me sick',
 'Ugly and talentless',
 'How attractive is she ',
 'I think there nose massive on screen',
 'Obesity on the rise',
 'Yuck - far too skinny',
 'How unattractive can one person be?',
 'Ugly and talentless twits',
 'You look radiant',
 'They make a brilliant actor on screen',
 'You go and rock those curves',
 'How minging can one person be?',
 'Fat and proud',
 'You look so fat in that picture',
 'Fat or skinny and proud',
 'How attractive']

## Load a model from hub

In [104]:
model = SetFitModel.from_pretrained('all-mpnet-base-v2', 
                                    num_labels=2, id2label=id2label, label2id=label2id)

config.json not found in HuggingFace Hub


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


## Create Trainer

In [110]:

MODEL_NAME = 'StatsGary/setfit-ft-online-abuse'

trainer = SetFitTrainer(
    model=model, 
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    loss_class=CosineSimilarityLoss, 
    batch_size=16,
    num_iterations=20,# The number of text pairs to generate for contrastive learning
    column_mapping={"text": "text", "label": "label"},
    num_epochs=2, 
    learning_rate=2e-5
)

In [111]:
trainer.train()

Applying column mapping to training dataset
***** Running training *****
  Num examples = 640
  Num epochs = 2
  Total optimization steps = 80
  Total train batch size = 16


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

## Evaluate the model

In [112]:
metrics = trainer.evaluate()

Applying column mapping to evaluation dataset
***** Running evaluation *****


In [113]:
print(metrics)

{'accuracy': 0.6666666666666666}


## Publish model to hub

In [114]:
from huggingface_hub import notebook_login

In [115]:
notebook_login()

In [92]:
!huggingface-cli whoami

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
StatsGary


In [116]:
!git config --global credential.helper store

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Push the model to the hub

In [117]:
trainer.push_to_hub(repo_path_or_name=MODEL_NAME)

/home/gary_hutson/anaconda3/envs/transformers/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:38: FutureWarning: Deprecated positional argument(s) used in 'push_to_hub': pass repo_path_or_name='StatsGary/setfit-ft-online-abuse', repo_url=None, commit_message='Add SetFit model', organization=None, private=None, api_endpoint=None, token=None, git_user=None, git_email=None, config=None, skip_lfs_files=False as keyword args. From version 0.12 passing these as positional arguments will result in an error,
  warnings.warn(
/home/gary_hutson/anaconda3/envs/transformers/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in 'push_to_hub': repo_path_or_name. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Upload file pytorch_model.bin:   0%|          | 32.0k/418M [00:00<?, ?B/s]

Upload file model_head.pkl: 100%|##########| 6.83k/6.83k [00:00<?, ?B/s]

remote: Scanning LFS files for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/StatsGary/setfit-ft-online-abuse
   2e3c07b..7fa215f  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'https://huggingface.co/StatsGary/setfit-ft-online-abuse/commit/7fa215f0bb8ab64c5536e59204af9249523fadbd'

## Load model from Hub for inference

In [118]:
model_infer = SetFitModel.from_pretrained(MODEL_NAME)

In [119]:
preds = model_infer(["you look horrible", "great performance", "fat people disgust me"]) 

In [137]:
import numpy as np 
results = list(np.array(preds))
print(results[0])

print(list(id2label.keys()))

if list(id2label.keys()) == 1:
    



1
[0, 1]


In [125]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model=MODEL_NAME)
classifier("you look absolutely horrible")

Some weights of the model checkpoint at StatsGary/setfit-ft-online-abuse were not used when initializing MPNetForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at StatsGary/setfit-ft-online-abuse and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream t

[{'label': 'LABEL_1', 'score': 0.5057770609855652}]

## Access via endpoint

Skip the below if you want to perform local inference. 

In [ ]:
import json
import requests as r

ENDPOINT_URL=""# url of your endpoint
HF_TOKEN="" #Your HF token

# payload samples
regular_payload = { "inputs": "Coming to The Rescue Got a unique problem? Not to worry: you can find a financial planner for every specialized need"}

# HTTP headers for authorization
headers= {
    "Authorization": f"Bearer {HF_TOKEN}",
    "Content-Type": "application/json"
}

# send request
response = r.post(ENDPOINT_URL, headers=headers, json=paramter_payload)
classified = response.json()

print(classified)